In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset
import pandas as pd

pd.options.display.max_columns= None

In [ ]:
dataset = load_dataset("AnikaBasu/CyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2956 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'text', 'instruction', 'output'],
        num_rows: 2956
    })
})

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,input,text,instruction,output
0,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
1,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
2,,Here is a classification that determines wheth...,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,,Here is a classification that determines wheth...,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,,Here is a classification that determines wheth...,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
def drop_columns(df, columns):
    return df.drop(columns=columns, inplace=False)

columns_to_drop = ['input', 'text']
df = drop_columns(df, columns_to_drop)


In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,"In other words #katandandre, your food was cra...",not_cyberbullying
2,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
df['output'].value_counts()

output
gender                 612
religion               562
other_cyberbullying    483
not_cyberbullying      434
age                    433
ethnicity              432
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df

,instruction,output
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,"In other words #katandandre, your food was cra...",not_cyberbullying
2,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
...,...,...
2951,U tell ppl its ok if someone ridiculed ur race...,ethnicity
2952,@SexiVixenFromMD you're right the lightskin/da...,ethnicity
2953,"Mad? >>RT @tayyoung_: FUCK OBAMA, dumb ass nigger",ethnicity
2954,@RealKayneWest fuck you you dumb nigger I woul...,ethnicity


In [ ]:
# Assign values to cyberbullying types
cyberbullying_type_mapping = {
    "age": 1,
    "ethnicity": 2,
    "gender": 3,
    "not_cyberbullying": 4,
    "other_cyberbullying": 5,
    "religion": 6
}
df['output'] = df['output'].map(cyberbullying_type_mapping)

In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",4
1,"In other words #katandandre, your food was cra...",4
2,Why is #aussietv so white? #MKR #theblock #ImA...,4
3,@XochitlSuckkks a classy whore? Or more red ve...,4
4,"@Jason_Gio meh. :P thanks for the heads up, b...",4


In [ ]:
def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Replace links with 'link'
    text = re.sub(r'http\S+|www\S+', 'link', text)

    # Remove mentions (words starting with @)
    text = re.sub(r'@\w+', '', text)

    #remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Punctuation and Special Characters
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Applying text preprocessing to the 'instruction' column in the DataFrame
df['instruction'] = df['instruction'].apply(preprocess_text)

print(df.head())

                                         instruction  output
0                            words food crapilicious       4
1                            words food crapilicious       4
2                                              white       4
3                   classy whore red velvet cupcakes       4
4  meh p thanks heads concerned another angry dud...       4


In [ ]:
# Function to remove emoticons from text
def remove_emoticons(text):
    return re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)

df['instruction'] = df['instruction'].apply(remove_emoticons)

In [ ]:

#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['instruction'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['instruction', 'tokens']].head())

                                         instruction  \
0                            words food crapilicious   
1                            words food crapilicious   
2                                              white   
3                   classy whore red velvet cupcakes   
4  meh p thanks heads concerned another angry dud...   

                                              tokens  
0                        [words, food, crapilicious]  
1                        [words, food, crapilicious]  
2                                            [white]  
3             [classy, whore, red, velvet, cupcakes]  
4  [meh, p, thanks, heads, concerned, another, an...  


In [ ]:
# NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

df['instruction'] = df['instruction'].apply(remove_stopwords)


In [ ]:
df['instruction']

0                                 words food crapilicious
1                                 words food crapilicious
2                                                   white
3                        classy whore red velvet cupcakes
4       meh p thanks heads concerned another angry dud...
                              ...                        
2951    u tell ppl ok someone ridiculed ur race cuz so...
2952    right battle dumb fuck cuz whites see nigger e...
2953                    mad rt fuck obama dumb ass nigger
2954    fuck dumb nigger would never stand fuckin show...
2955    pussy ass bitch dumb muthafuker like fuck u ni...
Name: instruction, Length: 2956, dtype: object

In [ ]:
#Split in train, test and validation
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#Making x_train and y_train columns
X_train = train_df['instruction']
y_train = train_df['output']

X_test = test_df['instruction']
y_test = test_df['output']

In [ ]:
#Import Libraries
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:
#Bert Model
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained("bert-large-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
# Tokenize and encode the text data using BERT
def encode_text(text):
  input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, padding=True, return_tensors="pt")
  with torch.no_grad():
    outputs = model(input_ids)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Apply encoding to the train and test data
X_train_encoded = [encode_text(text) for text in X_train]
X_test_encoded = [encode_text(text) for text in X_test]

In [ ]:
import numpy as np

# Define a function to encode text data in batches
def encode_text_batches(texts, batch_size=32):
    num_texts = len(texts)
    encoded_texts = []
    for i in range(0, num_texts, batch_size):
        batch_texts = texts[i:i+batch_size]
        encoded_batch = [encode_text(text) for text in batch_texts]
        encoded_texts.extend(encoded_batch)
        print(f"Encoded {len(encoded_texts)} out of {num_texts} texts")
    return encoded_texts

# Define the function to encode a single text using BERT
def encode_text(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Split the train and test data into batches and encode them
batch_size = 32
X_train_encoded_batches = encode_text_batches(X_train, batch_size=batch_size)
X_test_encoded_batches = encode_text_batches(X_test, batch_size=batch_size)

Encoded 32 out of 2364 texts
Encoded 64 out of 2364 texts
Encoded 96 out of 2364 texts
Encoded 128 out of 2364 texts
Encoded 160 out of 2364 texts
Encoded 192 out of 2364 texts
Encoded 224 out of 2364 texts
Encoded 256 out of 2364 texts
Encoded 288 out of 2364 texts
Encoded 320 out of 2364 texts
Encoded 352 out of 2364 texts
Encoded 384 out of 2364 texts
Encoded 416 out of 2364 texts
Encoded 448 out of 2364 texts
Encoded 480 out of 2364 texts
Encoded 512 out of 2364 texts
Encoded 544 out of 2364 texts
Encoded 576 out of 2364 texts
Encoded 608 out of 2364 texts
Encoded 640 out of 2364 texts
Encoded 672 out of 2364 texts
Encoded 704 out of 2364 texts
Encoded 736 out of 2364 texts
Encoded 768 out of 2364 texts
Encoded 800 out of 2364 texts
Encoded 832 out of 2364 texts
Encoded 864 out of 2364 texts
Encoded 896 out of 2364 texts
Encoded 928 out of 2364 texts
Encoded 960 out of 2364 texts
Encoded 992 out of 2364 texts
Encoded 1024 out of 2364 texts
Encoded 1056 out of 2364 texts
Encoded 108

In [ ]:
# Train Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_encoded_batches, y_train)  # Use X_train_encoded_batches instead of X_train_encoded

# Make predictions on train set
logreg_train_preds = logreg_model.predict(X_train_encoded_batches)  # Use X_train_encoded_batches instead of X_train_encoded

# Evaluate Logistic Regression model on train set
logreg_train_scores = {
    "accuracy": accuracy_score(y_train, logreg_train_preds),
    "precision": precision_score(y_train, logreg_train_preds, average='weighted'),
    "recall": recall_score(y_train, logreg_train_preds, average='weighted'),
    "f1": f1_score(y_train, logreg_train_preds, average='weighted')
}
print("Logistic Regression Scores:")
print("Accuracy:", logreg_train_scores["accuracy"])
print("Precision:", logreg_train_scores["precision"])
print("Recall:", logreg_train_scores["recall"])
print("F1-Score:", logreg_train_scores["f1"])
print()


Logistic Regression Scores:
Accuracy: 0.9851945854483926
Precision: 0.9851471441868421
Recall: 0.9851945854483926
F1-Score: 0.9851607922114298



In [ ]:
  # Train Logistic Regression model - TestData
logreg_model1 = LogisticRegression(max_iter=1000)
logreg_model1.fit(X_test_encoded_batches, y_test)  # Use X_test_encoded_batches instead of X_test_encoded

# Make predictions on test set
logreg_test_preds = logreg_model1.predict(X_test_encoded_batches)  # Use X_test_encoded_batches instead of X_test_encoded

# Evaluate Logistic Regression model on test set
logreg_test_scores = {
    "accuracy": accuracy_score(y_test, logreg_test_preds),
    "precision": precision_score(y_test, logreg_test_preds, average='weighted'),
    "recall": recall_score(y_test, logreg_test_preds, average='weighted'),
    "f1": f1_score(y_test, logreg_test_preds, average='weighted')
}
print("Logistic Regression Scores:")
print("Accuracy:", logreg_test_scores["accuracy"])
print("Precision:", logreg_test_scores["precision"])
print("Recall:", logreg_test_scores["recall"])
print("F1-Score:", logreg_test_scores["f1"])
print()


Logistic Regression Scores:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0



In [ ]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X_train_encoded_batches, y_train)

# Make predictions on train set
svm_train_preds = svm_model.predict(X_train_encoded_batches)

# Evaluate SVM model on train set
svm_train_scores = {
    "accuracy": accuracy_score(y_train, svm_train_preds),
    "precision": precision_score(y_train, svm_train_preds, average='weighted'),
    "recall": recall_score(y_train, svm_train_preds, average='weighted'),
    "f1": f1_score(y_train, svm_train_preds, average='weighted')
}
print("SVM Scores:")
print("Accuracy:", svm_train_scores["accuracy"])
print("Precision:", svm_train_scores["precision"])
print("Recall:", svm_train_scores["recall"])
print("F1-Score:", svm_train_scores["f1"])
print()

SVM Scores:
Accuracy: 0.8697123519458545
Precision: 0.8723017414265178
Recall: 0.8697123519458545
F1-Score: 0.8701950849089523



In [ ]:
# Test SVM model
svm_model1 = SVC()
svm_model1.fit(X_test_encoded_batches, y_test)

# Make predictions on train set
svm_test_preds = svm_model1.predict(X_test_encoded_batches)

# Evaluate SVM model on train set
svm_test_scores = {
    "accuracy": accuracy_score(y_test, svm_test_preds),
    "precision": precision_score(y_test, svm_test_preds, average='weighted'),
    "recall": recall_score(y_test, svm_test_preds, average='weighted'),
    "f1": f1_score(y_test, svm_test_preds, average='weighted')
}
print("SVM Scores:")
print("Accuracy:", svm_test_scores["accuracy"])
print("Precision:", svm_test_scores["precision"])
print("Recall:", svm_test_scores["recall"])
print("F1-Score:", svm_test_scores["f1"])
print()

SVM Scores:
Accuracy: 0.918918918918919
Precision: 0.9187317034448088
Recall: 0.918918918918919
F1-Score: 0.9182492631761481



In [ ]:
# Train Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X_train_encoded_batches, y_train)

# Make predictions on train set
nn_train_preds = nn_model.predict(X_train_encoded_batches)

# Evaluate Neural Network model on train set
nn_train_scores = {
    "accuracy": accuracy_score(y_train, nn_train_preds),
    "precision": precision_score(y_train, nn_train_preds, average='weighted'),
    "recall": recall_score(y_train, nn_train_preds, average='weighted'),
    "f1": f1_score(y_train, nn_train_preds, average='weighted')
}
print("Neural Network Scores:")
print("Accuracy:", nn_train_scores["accuracy"])
print("Precision:", nn_train_scores["precision"])
print("Recall:", nn_train_scores["recall"])
print("F1-Score:", nn_train_scores["f1"])
print()

Neural Network Scores:
Accuracy: 0.9919627749576988
Precision: 0.9923263637096126
Recall: 0.9919627749576988
F1-Score: 0.9919935203974058



In [ ]:
# Test Neural Network model
nn_model1 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model1.fit(X_test_encoded_batches, y_test)

# Make predictions on train set
nn_test_preds = nn_model1.predict(X_test_encoded_batches)

# Evaluate Neural Network model on train set
nn_test_scores = {
    "accuracy": accuracy_score(y_test, nn_test_preds),
    "precision": precision_score(y_test, nn_test_preds, average='weighted'),
    "recall": recall_score(y_test, nn_test_preds, average='weighted'),
    "f1": f1_score(y_test, nn_test_preds, average='weighted')
}
print("Neural Network Scores:")
print("Accuracy:", nn_test_scores["accuracy"])
print("Precision:", nn_test_scores["precision"])
print("Recall:", nn_test_scores["recall"])
print("F1-Score:", nn_test_scores["f1"])
print()

Neural Network Scores:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0



In [ ]:
import numpy as np
# # Reshape X_train_encoded to add another dimension
X_train_encoded_reshaped = np.expand_dims(X_train_encoded_batches, axis=-1)

# Reshape X_train_encoded to remove the extra dimension
X_train_encoded_reshaped = np.array(X_train_encoded_batches)

In [ ]:
# Train LSTM model
lstm_model = Sequential()
lstm_model.add(Dense(64, input_shape=(1024,), activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_encoded_reshaped, y_train, epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X_train_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate LSTM model on train set
lstm_train_scores = {
    "accuracy": accuracy_score(y_train, lstm_train_preds),
    "precision": precision_score(y_train, lstm_train_preds, average='weighted'),
    "recall": recall_score(y_train, lstm_train_preds, average='weighted'),
    "f1": f1_score(y_train, lstm_train_preds, average='weighted')
}
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores["accuracy"])
print("Precision:", lstm_train_scores["precision"])
print("Recall:", lstm_train_scores["recall"])
print("F1-Score:", lstm_train_scores["f1"])
print()


Epoch 1/10
74/74 [==============================] - 2s 3ms/step - loss: -124.4059 - accuracy: 0.1485
Epoch 2/10
74/74 [==============================] - 0s 3ms/step - loss: -601.1481 - accuracy: 0.1485
Epoch 3/10
74/74 [==============================] - 0s 3ms/step - loss: -1479.6379 - accuracy: 0.1485
Epoch 4/10
74/74 [==============================] - 0s 3ms/step - loss: -2784.0420 - accuracy: 0.1485
Epoch 5/10
74/74 [==============================] - 0s 3ms/step - loss: -4518.3672 - accuracy: 0.1485
Epoch 6/10
74/74 [==============================] - 0s 3ms/step - loss: -6665.7661 - accuracy: 0.1485
Epoch 7/10
74/74 [==============================] - 0s 3ms/step - loss: -9236.8516 - accuracy: 0.1485
Epoch 8/10
74/74 [==============================] - 0s 3ms/step - loss: -12203.9180 - accuracy: 0.1485
Epoch 9/10
74/74 [==============================] - 0s 3ms/step - loss: -15569.2236 - accuracy: 0.1485
Epoch 10/10
74/74 [==============================] - 0s 2ms/step
LSTM Scores:
Accu

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Reshape X_train_encoded to remove the extra dimension
X_test_encoded_reshaped = np.array(X_test_encoded_batches)

In [ ]:
# Test LSTM model
lstm_model1 = Sequential()
lstm_model1.add(Dense(64, input_shape=(1024,), activation='relu'))
lstm_model1.add(Dense(1, activation='sigmoid'))
lstm_model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model1.fit(X_test_encoded_reshaped, y_test, epochs=10, batch_size=32)

# Make predictions on train set
lstm_test_preds = (lstm_model1.predict(X_test_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate LSTM model on train set
lstm_test_scores = {
    "accuracy": accuracy_score(y_test, lstm_test_preds),
    "precision": precision_score(y_test, lstm_test_preds, average='weighted'),
    "recall": recall_score(y_test, lstm_test_preds, average='weighted'),
    "f1": f1_score(y_test, lstm_test_preds, average='weighted')
}
print("LSTM Scores:")
print("Accuracy:", lstm_test_scores["accuracy"])
print("Precision:", lstm_test_scores["precision"])
print("Recall:", lstm_test_scores["recall"])
print("F1-Score:", lstm_test_scores["f1"])
print()

Epoch 1/10
19/19 [==============================] - 1s 3ms/step - loss: -18.3688 - accuracy: 0.1351
Epoch 2/10
19/19 [==============================] - 0s 3ms/step - loss: -67.0611 - accuracy: 0.1385
Epoch 3/10
19/19 [==============================] - 0s 3ms/step - loss: -129.4057 - accuracy: 0.1385
Epoch 4/10
19/19 [==============================] - 0s 3ms/step - loss: -210.5490 - accuracy: 0.1385
Epoch 5/10
19/19 [==============================] - 0s 4ms/step - loss: -310.2754 - accuracy: 0.1385
Epoch 6/10
19/19 [==============================] - 0s 3ms/step - loss: -430.9098 - accuracy: 0.1385
Epoch 7/10
19/19 [==============================] - 0s 4ms/step - loss: -573.6284 - accuracy: 0.1385
Epoch 8/10
19/19 [==============================] - 0s 3ms/step - loss: -735.0046 - accuracy: 0.1385
Epoch 9/10
19/19 [==============================] - 0s 3ms/step - loss: -921.9592 - accuracy: 0.1385
Epoch 10/10
19/19 [==============================] - 0s 2ms/step
LSTM Scores:
Accuracy: 0.138

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
